In [21]:
import IPython.display as display
import pandas as pd


def draw(view):
    display.display(display.HTML(view))

def draw_cols(cols, with_refs=False):
    col_len = 100 // len(cols)
    draw(f'''
        <style>
            .qq {{
                word-break: keep-all;
                white-space: pre-wrap;
            }}
        </style>

        <div style="border:1px solid grey;padding:5px;border-radius:8px;">
            <div>ID: ID &nbsp Q: {i}</div>
            <h5 style="font-weight: bold;">{question}</h5>
            <div style="display:flex; align-items:top;">'''
                    + ''.join([
                                f'<div style="width: {col_len}%; padding:10px;"> <div style="text-align:center;color:blue;">{title}</div> <pre class="qq">{text}</pre></div>'
                                for title, text in cols
                                ])
            + '</div>'
            #+ str(extract_top_words(cols[1][1], 5))
            + (f'''<br><br>{"="*100}<br><br>
                 <pre>REFERENCES:<br><br>{"<br>".join(x["references_raw"])}
                 </pre>''' if with_refs else '')
        + '</div>')
    draw("<br>")

# Generate corpus from _data/training_albert-light.json :
# cat _data/training_albert-light.json  |  jq 'map(select(.answer != "nan")) | map({query:.query, prompt:.prompt, answer: .answer})' > notebooks/albert-light.json
corpus = pd.read_json("albert-light.json")

missed_patterns = [
    "ne fournissent pas",
    "a pas d'éléments de contexte",
    "pas possible de répondre",
    "peux pas répondre à cette question",
    "je ne sais pas",
    "je ne peux pas",
]


missed = corpus.apply(lambda x: any([q in x["answer"] for q in missed_patterns]), axis=1).sum()

print(f'''
total: {len(corpus)}
total negatif: {missed}
''')


for i, x in corpus.iterrows():
    question = x["query"]
    cols = [
        ("prompt", x["prompt"]),
        ("answer", x["answer"]),
    ]

    draw_cols(cols, with_refs=False)

    if i > 100:
        break


total: 1000
total negatif: 135

